In [1]:
import pandas as pd
import numpy as np

## Schritt 0 – Daten laden & einen ersten Blick werfen

In [2]:
# 0.1  Pfad zu deiner Datei anpassen, falls nötig
csv_path = "/Users/ilyaskaracabey/Desktop/Enviroments/AIS/data/processed/bearing/ais_with_distance.csv"

# 0.2  Einlesen
df = pd.read_csv(csv_path)

# 0.3  Grundinfos anzeigen
print("Zeilen:", len(df))
print("Spalten:", df.columns.tolist())
df.head()

Zeilen: 977539
Spalten: ['# Timestamp', 'MMSI', 'Latitude', 'Longitude', 'Destination', 'bearing', 'bearing_rate', 'dist_m']


,# Timestamp,MMSI,Latitude,Longitude,Destination,bearing,bearing_rate,dist_m
0,2022-10-01 00:00:20,229344000,54.530143,11.356242,LTKLJ,141.098699,0.000000,6930.459417
1,2022-10-01 00:00:40,229344000,54.529653,11.357865,LTKLJ,140.711241,-0.019373,7039.053050
2,2022-10-01 00:01:00,229344000,54.529249,11.359166,LTKLJ,140.414996,-0.014812,7127.190917
3,2022-10-01 00:01:20,229344000,54.528824,11.360489,LTKLJ,140.129870,-0.014256,7218.308615
4,2022-10-01 00:01:40,229344000,54.528394,11.361816,LTKLJ,139.852954,-0.013846,7310.139577


In [3]:
df["bearing_rate"]

0         0.000000
1        -0.019373
2        -0.014812
3        -0.014256
4        -0.013846
            ...   
977534    0.003647
977535    0.003771
977536    0.004075
977537    0.003972
977538    0.003842
Name: bearing_rate, Length: 977539, dtype: float64

## Schritt 1 – Raster‐Parameter setzen und die Daten bin-vorbereiten


In [4]:
# Schritt 1 ────────────────────────────────────────────────
import numpy as np
import pandas as pd

# 1.1  Raster-Parameter (gern später anpassen)
AZ_BIN_DEG = 5                       # Bearing-Bin-Breite: 5°
RATE_EDGES = [0, .01, .03, .1, .3, 1, 3, 10]   # |Bearing-Rate|-Bins in °/s

R_MAX_M  = 20_000                    # Max-Radius für später
R_STEP_M = 500                       # Distanz-Bin-Breite

# 1.2  Bearing in Azimut-Bins einteilen
df["bearing_bin"] = (df["bearing"] // AZ_BIN_DEG) * AZ_BIN_DEG

# 1.3  Absoluten Bearing-Rate-Wert und Rate-Bin bestimmen
df["bearing_rate_abs"] = df["bearing_rate"].abs()
df["bearing_rate_bin"] = pd.cut(df["bearing_rate_abs"],
                        bins=RATE_EDGES,
                        right=False,
                        labels=RATE_EDGES[:-1]).astype(float)

# 1.4  Kurzer Check
print("Eindeutige theta_bin-Werte:", sorted(df["bearing_bin"].unique())[:10], "...")
print("Eindeutige rate_bin-Werte :", sorted(df["bearing_rate_bin"].dropna().unique())[:10], "...")
df.head()

Eindeutige theta_bin-Werte: [np.float64(0.0), np.float64(5.0), np.float64(10.0), np.float64(15.0), np.float64(20.0), np.float64(25.0), np.float64(30.0), np.float64(35.0), np.float64(40.0), np.float64(45.0)] ...
Eindeutige rate_bin-Werte : [np.float64(0.0), np.float64(0.01), np.float64(0.03), np.float64(0.1), np.float64(0.3), np.float64(1.0), np.float64(3.0)] ...


,# Timestamp,MMSI,Latitude,Longitude,Destination,bearing,bearing_rate,dist_m,bearing_bin,bearing_rate_abs,bearing_rate_bin
0,2022-10-01 00:00:20,229344000,54.530143,11.356242,LTKLJ,141.098699,0.000000,6930.459417,140.0,0.000000,0.00
1,2022-10-01 00:00:40,229344000,54.529653,11.357865,LTKLJ,140.711241,-0.019373,7039.053050,140.0,0.019373,0.01
2,2022-10-01 00:01:00,229344000,54.529249,11.359166,LTKLJ,140.414996,-0.014812,7127.190917,140.0,0.014812,0.01
3,2022-10-01 00:01:20,229344000,54.528824,11.360489,LTKLJ,140.129870,-0.014256,7218.308615,140.0,0.014256,0.01
4,2022-10-01 00:01:40,229344000,54.528394,11.361816,LTKLJ,139.852954,-0.013846,7310.139577,135.0,0.013846,0.01


## Schritt 2 – Distanz in 500-Meter-Bins packen


In [5]:
# Schritt 2 ────────────────────────────────────────────────
import numpy as np

# 2.1  Distanz-Bin-Zentren (rangeVec) anlegen
range_vec = np.arange(0, R_MAX_M, R_STEP_M) + R_STEP_M / 2   # z. B. 250, 750, …

# 2.2  jedem Datenpunkt einen Distanz-Bin-Index zuweisen
df["dist_bin"] = (df["dist_m"] // R_STEP_M).clip(upper=len(range_vec)-1).astype(int)

# 2.3  Check
print("range_vec Länge:", len(range_vec), "erste Werte:", range_vec[:5])
print("dist_bin unique:", sorted(df["dist_bin"].unique())[:10], "...")
df.head()

range_vec Länge: 40 erste Werte: [ 250.  750. 1250. 1750. 2250.]
dist_bin unique: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)] ...


,# Timestamp,MMSI,Latitude,Longitude,Destination,bearing,bearing_rate,dist_m,bearing_bin,bearing_rate_abs,bearing_rate_bin,dist_bin
0,2022-10-01 00:00:20,229344000,54.530143,11.356242,LTKLJ,141.098699,0.000000,6930.459417,140.0,0.000000,0.00,13
1,2022-10-01 00:00:40,229344000,54.529653,11.357865,LTKLJ,140.711241,-0.019373,7039.053050,140.0,0.019373,0.01,14
2,2022-10-01 00:01:00,229344000,54.529249,11.359166,LTKLJ,140.414996,-0.014812,7127.190917,140.0,0.014812,0.01,14
3,2022-10-01 00:01:20,229344000,54.528824,11.360489,LTKLJ,140.129870,-0.014256,7218.308615,140.0,0.014256,0.01,14
4,2022-10-01 00:01:40,229344000,54.528394,11.361816,LTKLJ,139.852954,-0.013846,7310.139577,135.0,0.013846,0.01,14


## Schritt 3

In [6]:
# Schritt 3 ────────────────────────────────────────────────
# 3-D-Histogramm (bearing_bin × bearing_rate_bin × dist_bin)

import numpy as np
import pandas as pd   # für pd.isna in der Schleife

# 3.1  Anzahl Bins je Achse
AZ_BINS   = 360 // AZ_BIN_DEG            # z. B. 72
RATE_BINS = len(RATE_EDGES) - 1          # z. B. 7
R_BINS    = len(range_vec)               # z. B. 40

# 3.2  Leeres 3-D-Array
hist = np.zeros((AZ_BINS, RATE_BINS, R_BINS), dtype=int)

# 3.3  Gruppen zählen
grp_counts = (
    df.groupby(["bearing_bin", "bearing_rate_bin", "dist_bin"])
      .size()
)

# Mapping von Rate-Bin-Wert (linker Rand) zum Index 0…RATE_BINS-1
rate_val_to_idx = {v: i for i, v in enumerate(RATE_EDGES[:-1])}

# 3.4  Counts einsortieren
for (be_bin, ra_bin, dist_bin), cnt in grp_counts.items():
    if pd.isna(ra_bin):
        continue                # Zeilen ohne gültigen Rate-Bin überspringen
    be_i = int(be_bin / AZ_BIN_DEG)
    ra_i = rate_val_to_idx[ra_bin]
    hist[be_i, ra_i, int(dist_bin)] = cnt

# 3.5  Kurzer Check
print("hist shape (az, rate, dist):", hist.shape)
print("Summe aller Counts        :", hist.sum())

hist shape (az, rate, dist): (72, 7, 40)
Summe aller Counts        : 977539


## Schritt 4

In [7]:
# Schritt 4 ────────────────────────────────────────────────
import numpy as np

# 4.1  Zeilensummen über die Distanz-Achse
row_sums = hist.sum(axis=2, keepdims=True)

# 4.2  Division – nur dort, wo Summe > 0
prob_cube = np.divide(hist, row_sums, where=row_sums != 0)

# 4.3  Kontrolle
non_empty = np.count_nonzero(row_sums.squeeze() > 0)
print("Nicht-leere (bearing_bin (θ), bearing_rate_bin (ω))-Paare:", non_empty, "von", row_sums.size)
print("Beispiel-Summe erster aktiver Bin:",
      prob_cube[row_sums.squeeze() > 0][0].sum())


Nicht-leere (bearing_bin (θ), bearing_rate_bin (ω))-Paare: 403 von 504
Beispiel-Summe erster aktiver Bin: 1.0


## Schritt 5 – Lookup-Tabelle speichern & Runtime-Funktion bereitstellen


In [8]:
import pickle, os, json

# 5.1  Sammle alles in ein Dict
lut = {
    "params": {
        "az_bin_deg": AZ_BIN_DEG,
        "rate_edges": RATE_EDGES,         # list
        "range_vec":  range_vec.tolist(), # list
    },
    "prob_cube": prob_cube.astype("float32")  # 4× kleiner
}

# 5.2  Pfad
out_pkl = "data/processed/bearing/range_lut.pkl"
os.makedirs(os.path.dirname(out_pkl), exist_ok=True)

with open(out_pkl, "wb") as f:
    pickle.dump(lut, f)

print("✅ Lookup-Table gespeichert:", out_pkl)

✅ Lookup-Table gespeichert: data/processed/bearing/range_lut.pkl


## Schritt 6

In [9]:
import numpy as np, pickle

def load_range_lut(path="data/processed/bearing/range_lut.pkl"):
    with open(path, "rb") as f:
        return pickle.load(f)

def lookup_range_pdf(theta, omega, lut):
    p       = lut["params"]
    cube    = lut["prob_cube"]

    # 1) Bearing-Bin
    th_i = int((theta % 360) // p["az_bin_deg"])

    # 2) Rate-Bin (Absolutwert)
    abs_ω = abs(omega)
    edges = p["rate_edges"]
    ra_i  = np.searchsorted(edges, abs_ω, side="right") - 1
    if ra_i < 0 or ra_i >= len(edges)-1:
        return None, None          # außerhalb Raster

    pdf = cube[th_i, ra_i, :]
    if pdf.sum() == 0:
        return None, None          # kein Historien-Datensatz

    return np.array(p["range_vec"]), pdf

In [10]:
lut = load_range_lut()
range_vec, prob = lookup_range_pdf(theta=139.852954, omega=0.013846, lut=lut)

if prob is not None:
    print("Erwartete Distanz (m):", np.dot(range_vec, prob).round(1))
    print("Top-3 Wahrscheinlichkeiten:")
    for r, p in sorted(zip(range_vec, prob), key=lambda t: -t[1])[:5]:
        print(f"  R ≈ {r:6.0f} m   P={p:.3f}")
else:
    print("Keine Historie für diese (θ,ω)-Kombination.")

Erwartete Distanz (m): 8330.2
Top-3 Wahrscheinlichkeiten:
  R ≈   8750 m   P=0.132
  R ≈   8250 m   P=0.128
  R ≈   9250 m   P=0.117
  R ≈   7750 m   P=0.109
  R ≈   9750 m   P=0.097


## Plot